In [1]:
from __future__ import print_function
import torch
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import torch.utils.data
import torch.nn.init as init
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import pyDOE
from scipy import stats
import sys
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn import preprocessing
from sklearn.svm import SVR
from collections import namedtuple
from sklearn.linear_model import ElasticNet
import sklearn
from sklearn.preprocessing import PolynomialFeatures

print( 'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

torch: 1.3.0+cpu cuda: False


## Start Dimensionality Reduction Related Code

## Load & Re-Scale Data Sets

In [2]:
""" Give the path to the training input files as the function argument, the function loads and rescales the initial data set"""
def load_rescale_data_sets(path):
    train_data = pd.read_csv(path, index_col=0).iloc[:,:-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_data = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_2_200Samples.csv'), index_col=0).iloc[:,:-1]
    cols = test_data.columns
    scalar = MinMaxScaler().fit(train_data)
    train_data = pd.DataFrame(scalar.transform (train_data)) 
    test_data = pd.DataFrame(scalar.transform (test_data)) 
    train_data.columns, test_data.columns = cols, cols
    return train_data, test_data

## Autoencoders

In [3]:
""" Class Autoencoders, which implements the Autoencoders Neural Networks for Dimensionality Reduction """
class AE(nn.Module):
    
    def __init__(self, x_dim, z_dim):
        super(AE, self).__init__()
        
        self.x_dim = x_dim
        self.z_dim = z_dim
        
        self.h1_dim = int (self.x_dim - (1/5) * (self.x_dim - self.z_dim))
        self.h2_dim = int (self.x_dim - (2/5) * (self.x_dim - self.z_dim))
        self.h3_dim = int (self.x_dim - (3/5) * (self.x_dim - self.z_dim))
        self.h4_dim = int (self.x_dim - (4/5) * (self.x_dim - self.z_dim))

        #encoder
        self.enc = nn.Sequential( nn.Linear(self.x_dim , self.h1_dim), nn.Tanh(),
                                nn.Linear(self.h1_dim , self.h2_dim), nn.Tanh(),
                                nn.Linear(self.h2_dim , self.h3_dim), nn.Tanh(),
                                nn.Linear(self.h3_dim , self.h4_dim), nn.Tanh())
        
        self.enc_exact = nn.Sequential( nn.Linear(self.h4_dim, self.z_dim) )
        
        #decoder
        
        self.dec = nn.Sequential(nn.Linear(self.z_dim , self.h4_dim), nn.Tanh(),
                                nn.Linear(self.h4_dim , self.h3_dim), nn.Tanh(),
                                nn.Linear(self.h3_dim , self.h2_dim), nn.Tanh(),
                                nn.Linear(self.h2_dim , self.h1_dim), nn.Tanh())
                                
        self.dec_exact = nn.Sequential( nn.Linear( self.h1_dim, self.x_dim ))


    def encode (self, x ):
        enc = self.enc(x.float())
        enc_exact = self.enc_exact(enc)
        return enc_exact


    def decode (self, z):
        dec = self.dec(z)
        dec_exact = self.dec_exact(dec)
        return dec_exact

    def forward(self, x):
        loss = 0
        criterion = nn.MSELoss()
        #encoder
        enc = self.encode(x)
        #decoder
        dec = self.decode(enc)
        loss += criterion(dec, x.float())  
        return loss, enc, dec

## Train and Test the AE

In [4]:
""" Functions to Implement the Training and Testing of AEs, Based on Methods in Autoencoder Class"""
def train(epoch, train_loader, train_data, batch_size, model):
    train_loss = 0
    epoch_loss = np.zeros(int(len (train_data) / batch_size ))
    clip, learning_rate, seed, print_every, save_every  = 10, 1e-3 , 100, 10, 10
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for batch_idx, (data) in enumerate(train_loader):
        
        data = Variable(data)
        #forward + backward + optimize
        optimizer.zero_grad()
        re_loss,_,_ = model(data)
        epoch_loss [batch_idx] = re_loss
        loss = re_loss
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        #printing
        if batch_idx % print_every == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)] \t Reconstruction Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                re_loss.data / batch_size))

            

        train_loss += loss.data
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))
    return epoch_loss, model
    
def test(epoch, test_loader, test_data, model):
    """uses test data to evaluate 
    likelihood of the model"""
    mean_re_loss = 0
    epoch_loss = np.zeros(len(test_data))
    for i, (data) in enumerate(test_loader):                                           
        data = Variable(data.reshape(1,-1))
        re_loss, _,_ = model(data)
        epoch_loss [i] = re_loss
        mean_re_loss += re_loss.data

    mean_re_loss /= len(test_loader.dataset)

    print('====> Test set loss: Reconstruction Loss = {:.4f} '.format(
        mean_re_loss))
    return epoch_loss

## Perform Dimensionality Reduction

In [5]:
""" This is the method that implements the dimensionality reduction based on Autoencoders """
def perform_dimensionality_reduction (hyper_paras, path):
    seed = 100
    # Change the 0.3 to 0.6 and 0.9 for 60 % and 90 % dimensionality reduction
    z_dim = int(50-0.3 * 50) # 30 % dimensionality reduction
    n_epochs, batch_size = hyper_paras
    train_data, test_data = load_rescale_data_sets(path)
    x_dim = train_data.shape[1]
    
    h1_dim = int (x_dim - (1/5) * (x_dim - z_dim))
    h2_dim = int (x_dim - (2/5) * (x_dim - z_dim))
    h3_dim = int (x_dim - (3/5) * (x_dim - z_dim))
    h4_dim = int (x_dim - (4/5) * (x_dim - z_dim))
    
    print (x_dim, h1_dim, h2_dim, h3_dim,h4_dim,z_dim)
    
    torch.manual_seed(seed)
    train_loader = torch.utils.data.DataLoader ( dataset = train_data.values ,  batch_size = batch_size , shuffle= True)
    test_loader = torch.utils.data.DataLoader (  dataset = test_data.values , shuffle= True)
    train_error = np.zeros([n_epochs , int(train_data.shape[0] / batch_size ) ])
    test_error = np.zeros([n_epochs , test_data.shape[0]])  
    for epoch in range(1, n_epochs + 1):
        #training + testing
        if (epoch==1): ## only for the first time, take the default model, all the next times in iteration, keep improving it
            model = AE(x_dim, z_dim)
            
        tr = train(epoch, train_loader, train_data, batch_size, model)
        train_error [epoch-1 , :] = tr [0]
        model = tr[1]
        te = test(epoch, test_loader, test_data, model)
        test_error [epoch-1 , :] = te [0]
            
    train_lat = [ model (Variable(torch.tensor(train_data.iloc[idx,:].values)).reshape(1,-1))[1] for idx in range(len(train_data)) ]
    test_lat = [ model (Variable(torch.tensor(test_data.iloc[idx,:].values)).reshape(1,-1))[1] for idx in range(len(test_data)) ]
    train_lat = pd.DataFrame(torch.cat(train_lat).cpu().detach().numpy())
    test_lat = pd.DataFrame(torch.cat(test_lat).cpu().detach().numpy())
    cols = []
    for i in range(train_lat.shape[1]):
        cols.append(str('Z'+str(i+1)))
    train_lat.columns = cols
    test_lat.columns = cols
    train_lat.to_csv('AEs_ELN_50D_30%_latent_training.csv') # Change the path here depending upon dimensionality and reduction
    test_lat.to_csv('AEs_ELN_50D_30%_latent_test.csv') # Same here
    return train_error, test_error

## End Dimensionality Reduction Related Code

## Load New Reduced Data Sets for all Test Cases

In [6]:
def load_f2(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_2_200Samples.csv')).iloc[:,-1] 
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f3(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_3_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f7(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_7_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f9(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_9_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f10(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-43]+str('Test_Data_Sets\\test_10_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f13(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-43]+str('Test_Data_Sets\\test_13_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f15(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-43]+str('Test_Data_Sets\\test_15_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f16(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-43]+str('Test_Data_Sets\\test_16_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f20(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_20_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

def load_f24(path,path_latent_train,path_latent_test):
    train_y = pd.read_csv(path).iloc[:,-1]
    # Change the Path Here depending upon the dimensionality {50D=200,100D=400,200D=800}
    test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_24_200Samples.csv')).iloc[:,-1]
    train = pd.read_csv(path_latent_train, index_col = 0)
    test = pd.read_csv(path_latent_test, index_col = 0)
    train ['Y'] = train_y
    test ['Y'] = test_y
    del train_y
    del test_y
    true = np.array(test['Y'])
    return train,test,true

##  Start Polynomial Surrogate Modelling Code

In [7]:
''' Elastic Net Regression '''
def elastic_net(train_data,test_data,hyper):
    alp, rat = hyper
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= np.power(10,alp) ,random_state=0 , l1_ratio=rat, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    return regr,pred

""" Generating Polynomial Features i.e., Function Basis """
def quadratic_polynomial (df):
    return pd.DataFrame(PolynomialFeatures(degree=2).fit_transform(df))

""" Quadratic Regression with Elastic Net Penalty"""
def polynomial(tr, te,hyper):
    f_original = tr['Y']
    temp1 = quadratic_polynomial (tr.iloc[:,:-1])
    temp2 = quadratic_polynomial (te.iloc[:,:-1])
    temp1 ['Y'] = f_original
    model_eln , pred_eln = elastic_net(temp1,temp2,hyper)
    return model_eln , pred_eln


""" Normalized Mean Absolute Error % """
def rmae(true, pred):
    return np.mean((abs(true-pred) / abs(true)) * 100)

""" This method implements and evaluates the Polynomial Surrogate Model with RMAE """
def surrogate_model(train_data,test_data,hyper,true):
    model_eln , pred_eln = polynomial (train_data,test_data,hyper)
    return rmae(true,pred_eln)

""" Implements all the surrogate models, i.e., for all test function, and returns the median of RMAE errors,
This median is used as the primary metric for Hyper-Parameters Optimization """
def perform_surrogate_modeling(paths, path_latent_train,path_latent_test,hyper):
    train_2, test_2, true_2 = load_f2(paths[0],path_latent_train,path_latent_test)
    rmae_2 = surrogate_model(train_2, test_2,hyper, true_2)
    
    train_3, test_3, true_3 = load_f3(paths[1],path_latent_train,path_latent_test)
    rmae_3 = surrogate_model(train_3, test_3,hyper, true_3)
    
    train_7, test_7, true_7 = load_f7(paths[2],path_latent_train,path_latent_test)
    rmae_7 = surrogate_model(train_7, test_7,hyper, true_7)
    
    train_9, test_9, true_9 = load_f9(paths[3],path_latent_train,path_latent_test)
    rmae_9 = surrogate_model(train_9, test_9,hyper, true_9)
    
    train_10, test_10, true_10 = load_f10(paths[4],path_latent_train,path_latent_test)
    rmae_10 = surrogate_model(train_10, test_10,hyper, true_10)
    
    train_13, test_13, true_13 = load_f13(paths[5],path_latent_train,path_latent_test)
    rmae_13 = surrogate_model(train_13, test_13,hyper, true_13)
    
    train_15, test_15, true_15 = load_f15(paths[6],path_latent_train,path_latent_test)
    rmae_15 = surrogate_model(train_15, test_15,hyper, true_15)
    
    train_16, test_16, true_16 = load_f16(paths[7],path_latent_train,path_latent_test)
    rmae_16 = surrogate_model(train_16, test_16,hyper, true_16)
    
    train_20, test_20, true_20 = load_f20(paths[8],path_latent_train,path_latent_test)
    rmae_20 = surrogate_model(train_20, test_20,hyper, true_20)
    
    train_24, test_24, true_24 = load_f24(paths[9],path_latent_train,path_latent_test)
    rmae_24 = surrogate_model(train_24, test_24,hyper, true_24)
    
    accuracy = [rmae_2,rmae_3,rmae_7,rmae_9,rmae_10,rmae_13,rmae_15,rmae_16,rmae_20,rmae_24]
    return accuracy

""" This is the function used for Hyper_Parameters_Optimization for both dimensionality reduction and surrogate modelling """
def hyper_parameters_optimization(hyper_dim,path_latent_train,path_latent_test,hyper):
    print ('Start Dimensionality Reduction:::')
    _ , _ = perform_dimensionality_reduction (hyper_dim, paths[0])
    print ('End Dimensionality Reduction:::')
    accuracy = perform_surrogate_modeling (paths,path_latent_train,path_latent_test,hyper)
    return accuracy

## End Surrogate Modelling Code

## Set Paths

In [8]:
# Change the Paths here depending upon the dimensionality {50D=1000,100D=2000,200D=4000} 
path_2 = "Data Generation\\50 D\\Training_Data_Sets\\train_2_1000Samples.csv"
path_3 = "Data Generation\\50 D\\Training_Data_Sets\\train_3_1000Samples.csv"
path_7 = "Data Generation\\50 D\\Training_Data_Sets\\train_7_1000Samples.csv"
path_9 = "Data Generation\\50 D\\Training_Data_Sets\\train_9_1000Samples.csv"
path_10 = "Data Generation\\50 D\\Training_Data_Sets\\train_10_1000Samples.csv"
path_13 = "Data Generation\\50 D\\Training_Data_Sets\\train_13_1000Samples.csv"
path_15 = "Data Generation\\50 D\\Training_Data_Sets\\train_15_1000Samples.csv"
path_16 = "Data Generation\\50 D\\Training_Data_Sets\\train_16_1000Samples.csv"
path_20 = "Data Generation\\50 D\\Training_Data_Sets\\train_201000Samples.csv"
path_24 = "Data Generation\\50 D\\Training_Data_Sets\\train_241000Samples.csv"
# Change the Paths here depending upon the dimensionality {50D,100D,200D} and % of reduction{30,60,90} 
path_latent_train = "AEs_ELN_50D_30%_latent_training.csv"
path_latent_test = "AEs_ELN_50D_30%_latent_test.csv"
paths = [path_2,path_3,path_7,path_9,path_10,path_13,path_15,path_16,path_20,path_24]

## Run the Code

In [9]:
hyper_dim = [40,10]
hyper_surrogate = [-1.6460835015125062, 0.9981172056554478]
accuracy = hyper_parameters_optimization(hyper_dim,path_latent_train,path_latent_test,hyper_surrogate)
print ('The Median accuracy:::'+str(np.median(accuracy)))

Start Dimensionality Reduction:::
50 47 44 41 38 35
Train Epoch: 1 [0/1000 (0%)] 	 Reconstruction Loss: 0.033593
Train Epoch: 1 [100/1000 (10%)] 	 Reconstruction Loss: 0.023517
Train Epoch: 1 [200/1000 (20%)] 	 Reconstruction Loss: 0.011204
Train Epoch: 1 [300/1000 (30%)] 	 Reconstruction Loss: 0.009051
Train Epoch: 1 [400/1000 (40%)] 	 Reconstruction Loss: 0.008645
Train Epoch: 1 [500/1000 (50%)] 	 Reconstruction Loss: 0.008355
Train Epoch: 1 [600/1000 (60%)] 	 Reconstruction Loss: 0.008864
Train Epoch: 1 [700/1000 (70%)] 	 Reconstruction Loss: 0.007933
Train Epoch: 1 [800/1000 (80%)] 	 Reconstruction Loss: 0.008626
Train Epoch: 1 [900/1000 (90%)] 	 Reconstruction Loss: 0.008854
====> Epoch: 1 Average loss: 0.0119
====> Test set loss: Reconstruction Loss = 0.0852 
Train Epoch: 2 [0/1000 (0%)] 	 Reconstruction Loss: 0.008861
Train Epoch: 2 [100/1000 (10%)] 	 Reconstruction Loss: 0.008965
Train Epoch: 2 [200/1000 (20%)] 	 Reconstruction Loss: 0.008718
Train Epoch: 2 [300/1000 (30%)] 	 R

Train Epoch: 23 [300/1000 (30%)] 	 Reconstruction Loss: 0.005687
Train Epoch: 23 [400/1000 (40%)] 	 Reconstruction Loss: 0.005964
Train Epoch: 23 [500/1000 (50%)] 	 Reconstruction Loss: 0.005943
Train Epoch: 23 [600/1000 (60%)] 	 Reconstruction Loss: 0.006122
Train Epoch: 23 [700/1000 (70%)] 	 Reconstruction Loss: 0.005213
Train Epoch: 23 [800/1000 (80%)] 	 Reconstruction Loss: 0.005263
Train Epoch: 23 [900/1000 (90%)] 	 Reconstruction Loss: 0.005713
====> Epoch: 23 Average loss: 0.0058
====> Test set loss: Reconstruction Loss = 0.0633 
Train Epoch: 24 [0/1000 (0%)] 	 Reconstruction Loss: 0.005846
Train Epoch: 24 [100/1000 (10%)] 	 Reconstruction Loss: 0.005834
Train Epoch: 24 [200/1000 (20%)] 	 Reconstruction Loss: 0.005494
Train Epoch: 24 [300/1000 (30%)] 	 Reconstruction Loss: 0.006038
Train Epoch: 24 [400/1000 (40%)] 	 Reconstruction Loss: 0.006212
Train Epoch: 24 [500/1000 (50%)] 	 Reconstruction Loss: 0.005228
Train Epoch: 24 [600/1000 (60%)] 	 Reconstruction Loss: 0.005787
Train

C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.3756763220846936e+17, tolerance: 54476629894442.05
  positive)
C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11165648.13256216, tolerance: 644468.389133437
  positive)
C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 35077660.979091644, tolerance: 908658.0649560124
  positive)
C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWa

The Median accuracy:::23.903432881932744


## A Note on the Possible Values of the Hyper_Parameters

In [ ]:
""" A): Hyper-Parameters for Dimensionality Reduction::::

    Autoencoders takes 2 hyper parameters: Number of epochs and batch size respectively, 
    both of which are discrete variables.
    
    number of epochs, (Discrete Variable), Possible Values: {10,30,30,...,100}
    batch size: (Discrete Variable): Possible Values: {5,10,20,25,50,100}, 

"""
""" B): Hyper-Parameters for Polynomials::::

    Polynomials takes 2 hyper parameters: alpha and l1_ratio. Notably, both are float values
    https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html for more details
    
    Alpha (Continuous Variable, Ranges between -2 and 2, e.g., 0.1234
    l1_ratio (Continuous Variable, Ranges between 0 and 1, e.g., 0.97645
    
"""
model = AE(50, 35)
array = np.zeros((10000,35))
for i in range(len(array)):
    array [i] = model(Variable(torch.tensor(np.zeros(50))))[1].cpu().detach().numpy()
array1 = np.zeros((10000,35))
for i in range(len(array)):
    array1 [i] = model(Variable(torch.tensor(np.ones(50))))[1].cpu().detach().numpy()
np.mean(abs(np.mean(array1, 0)-np.mean(array, 0)))/4